<h1>Install and import dependencies</h1>

In [5]:
# %pip install mediapipe opencv-python
%pip uninstall opencv-python -y 

%pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/38/d2/3e8c13ffc37ca5ebc6f382b242b44acb43eb489042e1728407ac3904e72f/opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl.metadata
  Using cached opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl (38.1 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
%pip install notebook --upgrade

     ---------------------------------------- 0.0/133.1 kB ? eta -:--:--
     -------------------------------------- 133.1/133.1 kB 4.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/143.0 kB ? eta -:--:--
     ---------------------------------------- 143.0/143.0 kB ? eta 0:00:00
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.5/4.0 MB 10.5 MB/s eta 0:00:01
   ------------- -------------------------- 1.4/4.0 MB 17.7 MB/s eta 0:00:01
   --------------------- ------------------ 2.1/4.0 MB 17.0 MB/s eta 0:00:01
   ---------------------------- ----------- 2.9/4.0 MB 16.7 MB/s eta 0:00:01
   ------------------------------------ --- 3.6/4.0 MB 16.6 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 15.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/377.6 kB ? eta -:--:--
   --------------------------------------- 377.6/377.6 kB 11.5 MB/s eta 0:00:00
   --------------

In [2]:
import cv2
import csv
import time
import mediapipe
import numpy as np
import multiprocessing as mp
mp_drawing = mediapipe.solutions.drawing_utils
mp_pose = mediapipe.solutions.pose

print(cv2.__version__)
print(np.__version__)

4.8.1
1.26.0


In [5]:
# Video feed
cap = cv2.VideoCapture(0) # number that represents the webcam == 0
while cap.isOpened():
    ret, frame = cap.read() # Gets the current feed from the camera. ret isn't used, frame gives the actual videoframe
    cv2.imshow('Mediapip Feed', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):  # Checks if we try to close the feed by pressing 'q'
        break

cap.release()   # Releases the webcam
cv2.destroyAllWindows()

<h1>Determining Joints</h1>

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px">

<h1>Calculate Angles</h1>

In [3]:
def calculate_angle(a, b, c):
    a = np.array(a) # First
    b = np.array(b) # Midd
    c = np.array(c) # End

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360-angle
    
    return angle

In [13]:
# Video feed
cap = cv2.VideoCapture(0) # number that represents the webcam == 0

# Specify the file path where you want to save the CSV file
csv_file_path = 'elbow_angles.csv'

# Create an empty array to store angles
shape = (0, 1)
angles = np.zeros(shape)
times = np.zeros(shape)

# Initialize a variable to keep track of the start time
# start_time = time.time()

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        # current_time = time.time()

        # # Calculate the elapsed time in seconds
        # elapsed_time = current_time - start_time

        #     # Check if 1 second has passed
        # if elapsed_time >= 12.0:
        #     print("Broke loop after ", elapsed_time, " seconds")
        #     break  # Exit the loop after 1 second

        ret, frame = cap.read() # Gets the current feed from the camera. ret isn't used, frame gives the actual videoframe

        # Recolor image. frame comes in BGR, but we need it in RGB to predict the pose
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Make detection. Makes the predictions from the pose object to the image object, and stores it in results array
        results = pose.process(image)

        # Changes back to BGR, since openCV wants it in that format
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates
            shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
            elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y
            wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y

            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)

            # Save to angles array
            angles = np.vstack((angles, angle))
            # times = np.vstack((times, elapsed_time))

            # Visualize. Coordinates of the elbow is being multiplied by the webcam image resolution (640,480), to put the coordinate at the tip of the elbow
            cv2.putText(image, str(angle),
                        tuple(np.multiply(elbow, [640,480]).astype(int)),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                              )
                              
        except:
            pass

        # Render detections. Draws detections unto image. pose_landmarks gives all the predicted points and POSE_CONNECTIONS gives poit connections
        # draw_landmarks param: image as np array, list of landmarks, connections between landmarks, dots, lines
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow('Mediapip Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):  # Checks if we try to close the feed by pressing 'q'
            break

    cap.release()   # Releases the webcam
    cv2.destroyAllWindows()

    # Save to CSV
with open(csv_file_path, mode='w', newline='') as file:
    # Combine angle and time data into a list of lists
    # data = list(zip(angles, times))
    writer = csv.writer(file)
    writer.writerows(angles)

TypeError: Pose.__init__() got an unexpected keyword argument 'running_mode'